# **Fine Tune VGG16 on Image dataset**

# ****Load dataset****

In [ ]:
!pip install natsort

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import natsort
import pandas as pd 
import cv2

#specify path of image folder and label file
imageFolderPath='../input/custom-plantpathologydataset/custom_plantPathologyDataset/images/'
labelPath='../input/custom-plantpathologydataset/custom_plantPathologyDataset/label.csv'

#read images from a folder

sortImageDir=natsort.natsorted(os.listdir(imageFolderPath),reverse=False)
len(sortImageDir)
new_w, new_h=128, 128

resizeAll=[]
for file in sortImageDir:
    filePath=imageFolderPath+file
    frame=cv2.imread(filePath)/255
    resize = cv2.resize(frame, (new_w, new_h))
    resizeAll.append(resize)


In [ ]:
resizeAll=np.asarray(resizeAll)

temp=pd.read_csv(labelPath,header=None,skiprows=1,index_col=0)

label=temp.values

# **Splitting the model into training and test set**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(resizeAll, label, test_size=0.30, random_state=101,shuffle=True)

# **Loading the VGG16 model**

In [ ]:
	

from tensorflow.keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.applications.vgg16 import VGG16


# load model and specify a new input shape for images
new_input = Input(shape=(128, 128, 3))
baseModel = VGG16(include_top=False, input_tensor=new_input)


# **Freeze the layers of base model**

In [ ]:
# mark loaded layers as not trainable
for layer in baseModel.layers:
	  layer.trainable = False

# **Add some layers at the end of base model**

In [ ]:
# add new classifier layers
flat1 = Flatten()(baseModel.layers[-1].output)
class1 = Dense(32, activation='relu')(flat1)
output = Dense(4, activation='softmax')(class1)
# define new model
my_model = Model(inputs=baseModel.inputs, outputs=output)
# my_model.summary()

In [ ]:
for layer in my_model.layers:
    print(layer.name,'     ',layer.trainable)    

# **Compile and fit the model**

In [ ]:
# compile the keras model
from keras.optimizers import SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam
my_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy','mse','mae'])

# my_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [ ]:
my_history=my_model.fit(X_train, y_train, epochs=100, batch_size=8,verbose=1, validation_split=0.1)

# **Training history plots**

In [ ]:
# plot loss
from matplotlib import pyplot
pyplot.subplot(411)
pyplot.title('loss plot ')
pyplot.plot(my_history.history['loss'], label='train')
pyplot.plot(my_history.history['val_loss'], label='val')
pyplot.legend()
pyplot.show()

# plot acc during training
pyplot.subplot(412)
pyplot.title('accuracy plot')
pyplot.plot(my_history.history['accuracy'], label='train')
pyplot.plot(my_history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.show()

# plot acc during training
pyplot.subplot(412)
pyplot.title('mse plot')
pyplot.plot(my_history.history['mse'], label='train')
pyplot.plot(my_history.history['val_mse'], label='test')
pyplot.legend()
pyplot.show()

# plot acc during training
pyplot.subplot(412)
pyplot.title('mae plot')
pyplot.plot(my_history.history['mae'], label='train')
pyplot.plot(my_history.history['val_mae'], label='test')
pyplot.legend()
pyplot.show()

# **Model Evaluation**

In [ ]:
import keras

score = my_model.evaluate(X_test, y_test, verbose=1)
print("%s: %.2f%%" % (my_model.metrics_names[1], score[1]*100))